In [16]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
from tensorflow.keras.layers import Bidirectional
# set seeds for reproducability
# from tensorflow import set_random_seed
from numpy.random import seed
# set_random_seed(2)
# seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
# list(article_df.headline.values)

In [18]:
curr_dir =r'C:\Users\jv028u\OneDrive - Linde Group\Desktop\stats and ML\Projects'

all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir+ '\\' +filename)
        all_headlines.extend(list(article_df.headline.values))
        break

all_headlines = [h for h in all_headlines if h != "Unknown"]
len(all_headlines)

831

In [19]:
corpus = [] 
for headline in all_headlines:
    cleaned_text = [] 
    for char in headline: 
        if char not in string.punctuation:
            cleaned_text.append(char)
    txt = "".join(cleaned_text).lower()
    txt = txt.encode("utf8").decode("ascii", "ignore")
    corpus.append(txt) # Print first 10 entries from the corpus to verify
print(corpus[:10]) 

['finding an expansive view  of a forgotten people in niger', 'and now  the dreaded trump curse', 'venezuelas descent into dictatorship', 'stain permeates basketball blue blood', 'taking things for granted', 'the caged beast awakens', 'an everunfolding story', 'oreilly thrives as settlements add up', 'mouse infestation', 'divide in gop now threatens trump tax plan']


In [20]:
tokenizer=Tokenizer()
#each word is assigned a unique integer token (index).
tokenizer.fit_on_texts(corpus)
# print(corpus)
total_words=len(tokenizer.word_index)+1

# print(tokenizer.word_index)
input_sequences=[]
for line in corpus:
    # Convert the line to a list of tokens
    token_list=tokenizer.texts_to_sequences([line])[0]
    # print(token_list)  
    for i in range(1,len(token_list)):
        # Create N-gram sequences from the token list
        n_gram_sequence=token_list[:i+1]
        # print(n_gram_sequence)
        input_sequences.append(n_gram_sequence)  

In [21]:
# input_sequences

In [22]:
max_sequence_length=[]
for x in input_sequences:
    max_sequence_length.append(len(x))

max_len=max(max_sequence_length)

input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_len,padding='pre'))
predictor,label=input_sequences[:,:-1],input_sequences[:,-1]
label=ku.to_categorical(label,num_classes=total_words)

    

In [30]:
len(predictor)

4806

In [24]:
# input_len=max_len-1
# model=Sequential()

# model.add(Embedding(total_words,10,input_length=input_len))
# model.add(LSTM(100))
# model.add(Dropout(0.1))

# model.add(Dense(total_words,activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='adam')
# model.summary()



In [25]:
# model.fit(predictors,label,epochs=100,verbose=5)

In [31]:
model = Sequential()

model.add(Embedding(total_words, 128, input_length=max_len - 1))
model.add(Bidirectional(LSTM(256, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
# Summary of the model
# model.summary()
# Early stopping to avoid overfitting
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
model.fit(predictor, label, epochs=100, verbose=2, callbacks=[early_stop])


Epoch 1/100
151/151 - 30s - 198ms/step - loss: 7.3997
Epoch 2/100
151/151 - 16s - 108ms/step - loss: 6.6310
Epoch 3/100
151/151 - 16s - 105ms/step - loss: 5.8861
Epoch 4/100
151/151 - 25s - 168ms/step - loss: 4.8188
Epoch 5/100
151/151 - 41s - 269ms/step - loss: 3.6212
Epoch 6/100
151/151 - 25s - 167ms/step - loss: 2.4406
Epoch 7/100
151/151 - 23s - 149ms/step - loss: 1.7400
Epoch 8/100
151/151 - 23s - 153ms/step - loss: 1.1712
Epoch 9/100
151/151 - 38s - 253ms/step - loss: 0.8477
Epoch 10/100
151/151 - 23s - 154ms/step - loss: 0.6836
Epoch 11/100
151/151 - 22s - 148ms/step - loss: 0.5712
Epoch 12/100
151/151 - 39s - 259ms/step - loss: 0.5220
Epoch 13/100
151/151 - 22s - 145ms/step - loss: 0.4832
Epoch 14/100
151/151 - 22s - 144ms/step - loss: 0.4569
Epoch 15/100
151/151 - 22s - 145ms/step - loss: 0.4450
Epoch 16/100
151/151 - 39s - 258ms/step - loss: 0.4187
Epoch 17/100
151/151 - 24s - 159ms/step - loss: 0.4210
Epoch 18/100
151/151 - 39s - 255ms/step - loss: 0.5024
Epoch 19/100
151/15

In [32]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        # print(token_list)
        token_list = pad_sequences([token_list], maxlen=max_len-1, padding='pre')
        # print(token_list)
        predicted = np.argmax(model.predict(token_list), axis=-1)
        # print(predicted)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            # print(index,predicted)
            if index == predicted:
                # print(index)
                # print(predicted)
                # print(word)
                output_word = word
                # print(output_word)
                break
        seed_text += " "+output_word
    return seed_text.title()


In [33]:
print (generate_text("united states", 5, model, max_len))
print (generate_text("preident trump", 4, model, max_len))
print (generate_text("donald trump", 4, model, max_len))
print (generate_text("india and china", 4, model, max_len))
print (generate_text("new york", 4, model, max_len))
print (generate_text("science and technology", 5, model, max_len))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
United States More Spice Back In Real
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Preident Trump Cant Withhold Funding And
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Donald Trump And Silence Back In
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
India And China Cant Brings Moment Are
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
New York Today Health Plan Dies
1/1 ━━━━━━━━━━━━━━